# transformations

> Classes and functions that deal with transforming text during notebook export

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp markdown.transformations

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from fastcore.test import *

In [ ]:
#| hide
#| export
import io
import re
from typing import Callable, Iterable, Sequence

In [ ]:
# | export


class Transformer:
    """Base class for all content transformers."""

    def emit_before(self, stream: io.TextIOBase):
        """Implement this method on sub-classes to emit markdown \
            before the set of lines this transformer processes."""
        return

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        """Implement this method on sub-classes to modify the lines \
            this transformer processes."""
        return lines

    def emit_after(self, stream: io.TextIOBase):
        """Implement this method on sub-classes to emit markdown \
            after the set of lines this transformer processes."""
        return

In [ ]:
show_doc(Transformer.emit_before)

---

[source](https://github.com/spather/beetroot/blob/main/beetroot/markdown/transformations.py#L16){target="_blank" style="float:right; font-size:smaller"}

### Transformer.emit_before

>      Transformer.emit_before (stream:io.TextIOBase)

Implement this method on sub-classes to emit markdown             before the set of lines this transformer processes.

In [ ]:
show_doc(Transformer.process_lines)

---

[source](https://github.com/spather/beetroot/blob/main/beetroot/markdown/transformations.py#L21){target="_blank" style="float:right; font-size:smaller"}

### Transformer.process_lines

>      Transformer.process_lines (lines:Sequence[str])

Implement this method on sub-classes to modify the lines             this transformer processes.

In [ ]:
show_doc(Transformer.emit_after)

---

[source](https://github.com/spather/beetroot/blob/main/beetroot/markdown/transformations.py#L26){target="_blank" style="float:right; font-size:smaller"}

### Transformer.emit_after

>      Transformer.emit_after (stream:io.TextIOBase)

Implement this method on sub-classes to emit markdown             after the set of lines this transformer processes.

In [ ]:
# | export
class MultiTransformer(Transformer):
    def __init__(self, transformers: Iterable[Transformer]):
        # Store the passed in transformers as a list 
        # so that we can later call reverse() on it.
        self.transformers = list(transformers)

    def emit_before(self, stream: io.TextIOBase):
        # Emit all the before output from the transformers.
        # Do it in reversed order so that the first transformer's
        # before output appears closest to the (transformed) lines
        for transformer in reversed(self.transformers):
            transformer.emit_before(stream)

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        # Pass the source lines through all the transformers
        for transformer in self.transformers:
            lines = transformer.process_lines(lines)
        return lines

    def emit_after(self, stream: io.TextIOBase):
        # Emit the after output from the transformers
        for transformer in self.transformers:
            transformer.emit_after(stream)

In [ ]:
# | export

def emit_with_transformation(
    transformer: Transformer,
    lines: Sequence[str],
    emit_lines_func: Callable[[Sequence[str], io.TextIOBase], None],
    stream: io.TextIOBase,
):
    transformer.emit_before(stream)

    emit_lines_func(transformer.process_lines(lines), stream)

    transformer.emit_after(stream)


In [ ]:
# Tests for emit_with_transformations()
class TransformerA(Transformer):
    def emit_before(self, stream: io.TextIOBase):
        stream.write('before A\n')

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        return [
            line.replace('a', 'A')
            for line in lines 
        ]

    def emit_after(self, stream: io.TextIOBase):
        stream.write('after A\n')

class TransformerB(Transformer):
    def emit_before(self, stream: io.TextIOBase):
        stream.write('before B\n')

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        return [
            line.replace('b', 'B')
            for line in lines 
        ]

    def emit_after(self, stream: io.TextIOBase):
        stream.write('after B\n')

def dummy_emit_lines(lines: Sequence[str], stream: io.TextIOBase):
    for line in lines:
        stream.write(line)

lines = [
    'abCDefab\n',
    'ghiAbajkl\n'
    'mnoAAAAAABBBBBp\n'
]

stream = io.StringIO()

emit_with_transformation(
    transformer=MultiTransformer([TransformerA(), TransformerB()]),
    lines=lines,
    emit_lines_func=dummy_emit_lines,
    stream=stream
)

stream.seek(0)
output = stream.read()

expected = """\
before B
before A
ABCDefAB
ghiABAjkl
mnoAAAAAABBBBBp
after A
after B
"""

test_eq(output, expected)

In [ ]:
# | export
class ReplaceSingleDollarDelimiters(Transformer):
    """Transformer that replaces $ delimiters in inline latex\
        with \\\\( and \\\\)."""
    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        regex = r"(?<!\$)\$(?!\$)(.*?[^\\])\$(?!\$)(?!\w)"
        replacement = r"\\\\(\1\\\\)"

        return [re.sub(regex, replacement, line) for line in lines]

In [ ]:
# Test ReplaceSingleDollarDelimiters

input = [
    'This is an expression in which the delimiters will be replaced: $a + b$\n',
    'These block delimiters should not be replaced: $$a + b$$\n'
]

output = ReplaceSingleDollarDelimiters().process_lines(input)

expected = [
    r'This is an expression in which the delimiters will be replaced: \\(a + b\\)''\n',
    'These block delimiters should not be replaced: $$a + b$$\n'
]

test_eq(output, expected)


In [ ]:
# | export
class EscapeUnderscoresWithLatexMath(Transformer):
    """Transformer that replaces underscores within latex\
        math expressions with \_."""

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        regex_inline = r"(?<!\$)\$(?!\$)(.*?[^\\])\$(?!\$)"
        regex_block = r"\$\$([\s\S]*?)\$\$"

        # We want to handle cases where math expressions could be in a single
        # line or spread across multiple lines. So we'll join the lines with
        # a dummy token separator into a single string, perform the substitutions
        # and the split back into lines on the dummy token.
        dummy_token = 'DUMMY+TOKEN+DO+NOT+USE'
        text = dummy_token.join(lines)

        # Escaping underscores in block math expressions
        text = re.sub(
            regex_block, lambda match: re.sub(r"_", r"\_", match.group()), text
        )

        # Escaping underscores in inline math expressions
        text = re.sub(
            regex_inline, lambda match: re.sub(r"_", r"\_", match.group()), text
        )

        processed_lines = text.split(dummy_token)
        return processed_lines

In [ ]:
# Test EscapeUnderscoresWithLatexMath

test_cases = [
    {
        'name': 'Escape underscores',
        'lines': ['$a_1 + b_1$\n'],
        'expected': ['$a\_1 + b\_1$\n'],
    },
    {
        'name': 'Escape underscores',
        'lines': ['$$y_1 - x_1$$\n'],
        'expected': ['$$y\_1 - x\_1$$\n'],
    },
    {
        'name': 'Leave alone',
        'lines': ['$a + b$ and $$y - x$$\n'],
        'expected': ['$a + b$ and $$y - x$$\n'],
    },
    {
        'name': 'A block with some newlines',
        'lines': [
            '$$\\begin{bmatrix}\n',
            '{x}_{1,1} & {x}_{1,2} & {x}_{1,3} & {x}_{1,4}\\\\\n',
            '{x}_{2,1} & {x}_{2,2} & {x}_{2,3} & {x}_{2,4}\\\\\n',
            '{x}_{3,1} & {x}_{3,2} & {x}_{3,3} & {x}_{3,4}\\\\\n',
            '{x}_{4,1} & {x}_{4,2} & {x}_{4,3} & {x}_{4,4}\\end{bmatrix}\n',
            '$$',
        ],
        'expected': [
            '$$\\begin{bmatrix}\n',
            '{x}\_{1,1} & {x}\_{1,2} & {x}\_{1,3} & {x}\_{1,4}\\\\\n',
            '{x}\_{2,1} & {x}\_{2,2} & {x}\_{2,3} & {x}\_{2,4}\\\\\n',
            '{x}\_{3,1} & {x}\_{3,2} & {x}\_{3,3} & {x}\_{3,4}\\\\\n',
            '{x}\_{4,1} & {x}\_{4,2} & {x}\_{4,3} & {x}\_{4,4}\\end{bmatrix}\n',
            '$$',
        ],
    },
    {
        'name': 'Expressions both in single lines and across lines',
        'lines': [
            '$a_1 + b_1$\n',
            '$$y_1 - x_1$$\n' '\n',
            '$$\\begin{bmatrix}\n',
            '{x}_{1,1} & {x}_{1,2} & {x}_{1,3} & {x}_{1,4}\\\\\n',
            '{x}_{2,1} & {x}_{2,2} & {x}_{2,3} & {x}_{2,4}\\\\\n',
            '{x}_{3,1} & {x}_{3,2} & {x}_{3,3} & {x}_{3,4}\\\\\n',
            '{x}_{4,1} & {x}_{4,2} & {x}_{4,3} & {x}_{4,4}\\end{bmatrix}\n',
            '$$',
        ],
        'expected': [
            '$a\_1 + b\_1$\n',
            '$$y\_1 - x\_1$$\n' '\n',
            '$$\\begin{bmatrix}\n',
            '{x}\_{1,1} & {x}\_{1,2} & {x}\_{1,3} & {x}\_{1,4}\\\\\n',
            '{x}\_{2,1} & {x}\_{2,2} & {x}\_{2,3} & {x}\_{2,4}\\\\\n',
            '{x}\_{3,1} & {x}\_{3,2} & {x}\_{3,3} & {x}\_{3,4}\\\\\n',
            '{x}\_{4,1} & {x}\_{4,2} & {x}\_{4,3} & {x}\_{4,4}\\end{bmatrix}\n',
            '$$',
        ],
    },
    {
        'name': 'Pathological case: unterminated multi-line block (should not change)',
        'lines': [
            '$$\\begin{bmatrix}\n',
            '{x}_{1,1} & {x}_{1,2} & {x}_{1,3} & {x}_{1,4}\\\\\n',
            '{x}_{2,1} & {x}_{2,2} & {x}_{2,3} & {x}_{2,4}\\\\\n',
            '{x}_{3,1} & {x}_{3,2} & {x}_{3,3} & {x}_{3,4}\\\\\n',
            '{x}_{4,1} & {x}_{4,2} & {x}_{4,3} & {x}_{4,4}\\end{bmatrix}\n',
        ],
        'expected': [
            '$$\\begin{bmatrix}\n',
            '{x}_{1,1} & {x}_{1,2} & {x}_{1,3} & {x}_{1,4}\\\\\n',
            '{x}_{2,1} & {x}_{2,2} & {x}_{2,3} & {x}_{2,4}\\\\\n',
            '{x}_{3,1} & {x}_{3,2} & {x}_{3,3} & {x}_{3,4}\\\\\n',
            '{x}_{4,1} & {x}_{4,2} & {x}_{4,3} & {x}_{4,4}\\end{bmatrix}\n',
        ],
    },
]

for tc in test_cases:
    name = tc['name']
    lines = tc['lines']
    expected = tc['expected']

    print(f"Case: {name}")
    output = EscapeUnderscoresWithLatexMath().process_lines(lines)
    test_eq(output, expected)

Case: Escape underscores
Case: Escape underscores
Case: Leave alone
Case: A block with some newlines
Case: Expressions both in single lines and across lines
Case: Pathological case: unterminated multi-line block (should not change)


In [ ]:
# | export
class Unindent(Transformer):
    """Transformer that removes leading indentation from a set\
        of lines. Will determine how far indented the first line \
        is and then remove \
        min(indentation of first non-empty line, indentation of current line) \
        for each subsequent line."""
    
    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        i = 0 
        for i, line in enumerate(lines):
            if line.strip() != '':
                break

        # i is now the index of the first line that is not all whitespace
        # or len(lines)-1 if all the lines were just whitespace.

        if i == len(lines) - 1 and line.strip() == '':
            # Everything was whitespace, just return it
            # unmodified
            return lines 
        
        # If we're here, i is the index of the first line that is not 
        # all whitespace. 

        first_line_indent = len(lines[i]) - len(lines[i].lstrip())

        processed_lines = []
        for line in lines[i:]:
            cur_indent = len(line) - len(line.lstrip())
            remove_count = min(first_line_indent, cur_indent)
            processed_lines.append(line[remove_count:])

        return processed_lines




In [ ]:
# Test Unindent

test_cases = [
    {
        'name': 'basic',
        'lines': [
            '    line 2\n',
            '      - line 2\n',
            '      - line 3\n',
            '    line 4\n',
            '      - line 5\n',
            'line 6\n',
        ],
        'expected': [
            'line 2\n',
            '  - line 2\n',
            '  - line 3\n',
            'line 4\n',
            '  - line 5\n',
            'line 6\n',
        ]
    },
    {
        'name': 'all lines are whitespace - leave unmodified',
        'lines': [
            '     \n',
            '\n',
            '  \n',
        ],
        'expected': [
            '     \n',
            '\n',
            '  \n',
        ]
    },
    {
        'name': 'leading whitespace lines are stripped',
        'lines': [
            '     \n',
            '\n',
            '    line 2\n',
            '      - line 2\n',
            '      - line 3\n',
            '    line 4\n',
            '      - line 5\n',
            'line 6\n',
        ],
        'expected': [
            'line 2\n',
            '  - line 2\n',
            '  - line 3\n',
            'line 4\n',
            '  - line 5\n',
            'line 6\n',
        ]
    }

]

unindent = Unindent()

for tc in test_cases:
    name = tc['name']
    lines = tc['lines']
    expected = tc['expected']

    print(f"Case: {name}")
    output = unindent.process_lines(lines)
    test_eq(output, expected)


Case: basic
Case: all lines are whitespace - leave unmodified
Case: leading whitespace lines are stripped


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()